# Because Cambridge is not "an African even-toed ungulate mammal"

This is a word perturbation analysis (originally reported in the [DESM](https://arxiv.org/abs/1602.01137v1.pdf) (2016) paper) to show how the model collects evidence on the aboutness of a document. The DESM models are more robust irrelevant terms. For example, when the word "giraffe" is replaced by the word "cambridge", the passage on giraffes is still scored low by the DESM for the query "cambridge" because it finds low supporting evidence from the other words in the passage. However, the DESM confuses the passage about Oxford to be relevant for the query "cambridge" because it detects a high number of similar words in the passage that frequently co-occur with the word "Cambridge".

The word embeddings data used in this demonstration can be downloaded from the following location,
https://www.microsoft.com/en-us/download/details.aspx?id=52597

In [1]:
import csv
import re
import math
from IPython.display import HTML, display

def cosine_sim(a, b):
    if a == None or b ==None:
        return 0
    
    d = dot_product(a, b);
    na = max(1e-20, norm(a));
    nb = max(1e-20, norm(b));
    return (d / (na * nb) + 1) / 2;

def dot_product(a, b):
    result = 0;
    for i in range(len(a)):
        result += (a[i] * b[i])
    return result;
    
def norm(a):
    return math.sqrt(dot_product(a, a))
    
def centroid(a):
    dim = len(a[0])
    num = len(a)
    b = []
    for i in range(dim):
        b.append(0)
        for j in range(num):
            b[i] += (a[j][i] / num)
    return b

def get_clean(s):
    return re.sub('[^0-9a-z\t]+', '', s.lower())
    
def get_pre_punc(s):
    c = s[0].lower()
    o = ord(c)
    if (o < ord('a') or o > ord('z')) and (o < ord('0') or o > ord('9')):
        return c
    else:
        return ""

def get_post_punc(s):
    c = s[len(s)-1].lower()
    o = ord(c)
    if (o < ord('a') or o > ord('z')) and (o < ord('0') or o > ord('9')):
        return c
    else:
        return ""
    
def get_font(x):
    x = (x + 1) / 2
    f = 2
    size = math.exp(f*x) * 10 / math.exp(f)
    x = min(1.7 - x, 1)
    f = 8
    color = hex(math.floor(math.exp(f*x) * 255 / math.exp(f)))[2:]
    #color = hex(math.floor(x * 255))[2:]
    if len(color) == 1: color = "0" + color
    return "size=\"{}\" color=\"#{}{}{}\"".format(size, color, color, color)
    #return "size=\"{}\" color=\"#ffffff\"".format(size)


query = "cambridge"
passage = [ "The city of Cambridge is a university city and the county town of Cambridgeshire, England. It lies in East Anglia, on the River Cam, about 50 miles (80 km) north of London. According to the United Kingdom Census 2011, its population was 123867 (including 24488 students). This makes Cambridge the second largest city in Cambridgeshire after Peterborough, and the 54th largest in the United Kingdom. There is archaeological evidence of settlement in the area during the Bronze Age and Roman times; under Viking rule Cambridge became an important trading centre. The first town charters were granted in the 12th century, although city status was not conferred until 1951.",
            "Oxford is a city in the South East region of England and the county town of Oxfordshire. With a population of 159994 it is the 52nd largest city in the United Kingdom, and one of the fastest growing and most ethnically diverse. Oxford has a broad economic base. Its industries include motor manufacturing, education, publishing and a large number of information technology and science-based businesses, some being academic offshoots. The city is known worldwide as the home of the University of Oxford, the oldest university in the English-speaking world. Buildings in Oxford demonstrate examples of every English architectural period since the arrival of the Saxons, including the mid-18th-century Radcliffe Camera. Oxford is known as the city of dreaming spires, a term coined by poet Matthew Arnold.",
            "The giraffe (Giraffa camelopardalis) is an African even-toed ungulate mammal, the tallest living terrestrial animal and the largest ruminant. Its species name refers to its camel-like shape and its leopard-like colouring. Its chief distinguishing characteristics are its extremely long neck and legs, its horn-like ossicones, and its distinctive coat patterns. It is classified under the family Giraffidae, along with its closest extant relative, the okapi. The nine subspecies are distinguished by their coat patterns. The giraffe’s scattered range extends from Chad in the north to South Africa in the south, and from Niger in the west to Somalia in the east. Giraffes usually inhabit savannas, grasslands, and open woodlands."]
passage.append(passage[2].replace("giraffe", "Cambridge").replace("Giraffe", "Cambridge"))
passage.append(passage[0].replace("Cambridge", "giraffe"))
doc_all = " ".join(passage)
words = re.sub('[^0-9a-z\t]+', ' ', doc_all.lower()).split()
seen = set()
vocab = [x for x in words if not (x in seen or seen.add(x))]

in_vec = {}
out_vec = {}

with open("data\\in.txt", mode='r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row[0] in vocab:
            in_vec[row[0]] = [float(row[i]) for i in range(1, 201)]

with open("data\\out.txt", mode='r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row[0] in vocab:
            out_vec[row[0]] = [float(row[i]) for i in range(1, 201)]
            
q_vec = centroid([in_vec[word] for word in query.split()])

display(HTML("<br /><br /><h3>Query: \"cambridge\"</h3><br /><br /><p style=\"border-left: 6px solid blue; background-color: #fafafa\"><br /><br />" + "<br /><br /></p><br /><br /><p style=\"border-left: 6px solid blue; background-color: #fafafa\"><br /><br />".join(" ".join("{}<font {}>{}</font>{}".format(get_pre_punc(word), get_font(cosine_sim(q_vec, out_vec.get(get_clean(word), None))), get_clean(word), get_post_punc(word)) for word in passage[i].split()) for i in range(5)) + "<br /><br /></p>"))
